# Car Price Prediction - ML Deployment Project

This project aims to develop a machine learninng model to predict the car prices. The data have been scrapped from autoscout website in 2022. Project steps are:

1) EDA and feature engineering
2) Machine learning algorithms
    - Multiple linear regression
    - Lasso regression
    - Random forest
    - XG-Boost
3) Final model determination and pickling the model
4) By using streamlit, creating a website to enable user to predict car prices
5) Deployment of the model via Streamlit and AWS EC2 
    

In [ ]:
import pandas as pd      
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

from scipy.stats import skew

from sklearn.model_selection import cross_validate
import warnings
warnings.filterwarnings('ignore')
plt.rcParams["figure.figsize"] = (7,4)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)
pd.options.display.float_format = '{:.3f}'.format

In [ ]:
df0 = pd.read_csv("final_scout_not_dummy.csv")
df = df0.copy()
df.shape

In [ ]:
df0.shape

In [ ]:
df = df0.copy()

# 1. EDA and Feature Engineering

In [ ]:
df.info()

In [ ]:
df.head(2)

In [ ]:
# check and drop duplicates
df.duplicated().sum()

In [ ]:
# drop duplicates
df.drop_duplicates(inplace=True)

In [ ]:
# check the shape
df.shape

In [ ]:
# check null values
df.isnull().sum()
# no null values

# 1.1 Categorical Variables

In [ ]:
df_object = df.select_dtypes(include="object")
df_object.columns

In [ ]:
# check the value counts for each feature to determine the features with high cardinality
for col in df_object:
    print(f"{col:<30}:", df[col].nunique())

# comfort&con, entertainemtn&media, extras and safety_sec should be dealt with.

## 1.1.1 make_model feature

In [ ]:
df.make_model.value_counts()

In [ ]:
# get the index of the make_models which have less than 100 samples, since the mdoel can't learn efficiently
ind = df[(df.make_model == "Renault Duster") | (df.make_model == "Audi A2")].index
ind

In [ ]:
# check shape before dropping
df.shape

In [ ]:
# drop them
df.drop(index=ind, inplace=True)

In [ ]:
# check updated shape
df.shape

In [ ]:
# visualize the number of samples for each make_model
ax = sns.countplot(data=df, x= "make_model");

ax.bar_label(ax.containers[0]);

## 1.1.2 body_type

In [ ]:
df.body_type.value_counts()

## 1.1.3 vat

In [ ]:
df.vat.value_counts()

## 1.1.4 Type 

In [ ]:
df.Type.value_counts()

## 1.1.5 Fuel

In [ ]:
df.Fuel.value_counts()

In [ ]:
df.groupby("Fuel")["price"].mean()

In [ ]:
ind_fuel = df[(df.Fuel == "LPG/CNG") | (df.Fuel == "Electric")].index
    

In [ ]:
df.drop(index= ind_fuel, inplace=True)

In [ ]:
df.Fuel.value_counts()

## 1.1.6 Comfort_Convenience


In [ ]:
df.Comfort_Convenience.nunique()

In [ ]:
premium = ["Electrical side mirrors", "Parking assist", "Air conditioning", "Hill Holder", "Power windows"]
premium_plus = ["Multi-function", "Navigation ", "Keyless central door lock", "Heads-up", "Massage seats", "heating", "Automatic climate control", "Heated"]

comfort_package = df['Comfort_Convenience'].apply(lambda sentence: "Premium Plus" if all(word in sentence for word in premium_plus) else ("Premium" if all(word in sentence for word in premium) else "Standard"))
comfort_package.value_counts()

# since there are only 63 cars in the category of premium plus, they will be also labelled as premiumn

In [ ]:
comfort_package.shape

In [ ]:
df['Comfort_Convenience'] = comfort_package
df['Comfort_Convenience'].value_counts()

In [ ]:
df['Comfort_Convenience'] = df['Comfort_Convenience'].replace("Premium Plus", "Premium")
df['Comfort_Convenience'].value_counts()

## 1.1.7 Entertainment_Media

In [ ]:
# if an auto contains digital or television in the entertainment feature, then they can be labelled
# as plus while the cars without these features can be labelled as standard
media_plus = ["Digital", "Television"]


entertainment = df['Entertainment_Media'].apply(lambda sentence: "Plus" if any(word in sentence for word in media_plus) else "Standard")
entertainment.value_counts()

In [ ]:
df['Entertainment_Media'] = entertainment
df['Entertainment_Media'].value_counts()

## 1.1.8 Extras

In [ ]:
# count the number of extras for each observation
df.Extras.apply(lambda x: len(x.split(',')))

In [ ]:
# convert the feature from object to numeric by taking the number of features into account
df["Extras"] = df.Extras.apply(lambda x: len(x.split(',')))
df["Extras"].value_counts()

## 1.1.9 Safety_Security

In [ ]:
premium = ["Tire pressure", "Traction control", "Daytime running lights", "LED Headlight", "Tire pressure"]
premium_plus = ["Emergency brake assistant", "Electronic stability control"]

safety = df['Safety_Security'].apply(lambda sentence: "Safety Premium Package" if any(word in sentence for word in premium) else ("Safety Premium Plus Package" if any(word in sentence for word in premium_plus) else "Safety Standard Package"))
safety.value_counts()

In [ ]:
df['Safety_Security'] = safety
df['Safety_Security'].value_counts()

## 1.1.10 Paint_Type

In [ ]:
df.Paint_Type.value_counts()

In [ ]:
df[df.Paint_Type == "Perl effect"].index

In [ ]:
df.drop(index = df[df.Paint_Type == "Perl effect"].index, inplace=True)

In [ ]:
df.Paint_Type.value_counts()

## 1.1.11 Upholstery_type

In [ ]:
df.Upholstery_type.value_counts()

## 1.1.12 Gearing_Type

In [ ]:
df.Gearing_Type.value_counts()

## 1.1.13 Drive_chain

In [ ]:
df.Drive_chain.value_counts()

In [ ]:
# we will drop this feature since the categories are distributed too imbalanced
df.drop("Drive_chain", axis=1, inplace=True)

# 1.2. Numerical Features

In [ ]:
df.select_dtypes(include="number").columns

In [ ]:
df.describe().T

In [ ]:
def show_distribution(col):
    
    '''
    
    This function will prints a Histogram and box plot which are graphical representations 
    for the frequency of numeric data values. It aims to describe the data and explore 
    the central tendency and variability before using advanced statistical analysis techniques. 
    
    '''
    # Get statistics
    from termcolor import colored

    print(colored('Statistical Calculations :', 'red', attrs=['bold']))
    print(colored('-'*26, 'red', attrs=['bold']))    
    min_val = col.min()
    max_val = col.max()
    mean_val = col.mean()
    med_val = col.median()
    mod_val = col.mode()[0]

    print(colored('Minimum:{:>7.2f}\nMean:{:>10.2f}\nMedian:{:>8.2f}\nMode:{:>10.2f}\nMaximum:{:>7.2f}\n'.format(min_val,
                                                                                             mean_val,
                                                                                             med_val,
                                                                                             mod_val,
                                                                                             max_val), 'blue', attrs=['bold']))
    
    # Create a figure for 2 subplots (2 rows, 1 column)
    fig, ax = plt.subplots(2, 1, figsize=(15, 15))

    # Plot the histogram   
    ax[0].hist(col, bins=30)
    ax[0].set_ylabel('Frequency', fontsize=10)

    # Add lines for the mean, median, and mode
    ax[0].axvline(x=min_val,  color='orange',     linestyle='dashed', linewidth=2, label='Minimum')
    ax[0].axvline(x=mean_val, color='lightgreen', linestyle='dashed', linewidth=2, label='Mean')
    ax[0].axvline(x=med_val,  color='cyan',       linestyle='dashed', linewidth=2, label='Median')
    ax[0].axvline(x=mod_val,  color='purple',     linestyle='dashed', linewidth=2, label='Mode')
    ax[0].axvline(x=max_val,  color='red',        linestyle='dashed', linewidth=2, label='Maximum')
    ax[0].legend(loc='upper right')

    # Plot the boxplot   
    medianprops = dict(linestyle='-', linewidth=3, color='m')
    boxprops=dict(linestyle='-', linewidth=1.5)
    meanprops={"marker":"d", "markerfacecolor":"blue", "markeredgecolor":"black", "markersize":"10"}
    flierprops={'marker': 'o', 'markersize': 8, 'markerfacecolor': 'fuchsia'}
    
    ax[1].boxplot(col, 
                  vert=False,
                  notch=True, 
                  patch_artist=False,
                  medianprops=medianprops,
                  flierprops=flierprops,
                  showmeans=True,
                  meanprops=meanprops)
    
    ax[1].set_xlabel('value', fontsize=10)
    

    # Add a title to the Figure
    fig.suptitle('Data Distribution', fontsize=20)

## 1.2.1 price

In [ ]:
# show_distribution(df.price)

In [ ]:
# check make & models with interactive plots

from ipywidgets import interact

def box_strip(model):
    sns.boxplot(data = df[df.make_model==model],
                x= "make_model",
                y= "price",
                palette='bright')
    
    sns.stripplot(data = df[df.make_model==model],
                x= "make_model",
                y= "price",
                palette='Set1')
model = df.make_model.unique()
interact(box_strip, model=model);

## 1.2.2 km

In [ ]:
# show_distribution(df.km)

In [ ]:
# over 150k can be dropped?

In [ ]:
df[df.km > 150000].shape

## 1.2.3 Gears

In [ ]:
# show_distribution(df.Gears)

## 1.2.4 age

In [ ]:
# show_distribution(df.age)

## 1.2.5 Previous_Owners

In [ ]:
# show_distribution(df.Previous_Owners)

## 1.2.6 hp_kW

In [ ]:
# show_distribution(df.hp_kW)

In [ ]:
df[df.hp_kW>200].shape

In [ ]:
# over 200 can be dropped?

## 1.2.7 Inspection_new

In [ ]:
df.Inspection_new.value_counts()

## 1.2.8 Displacement_cc

In [ ]:
# show_distribution(df.Displacement_cc)

In [ ]:
# drop the only auto with a displacement over 2480 cc
df.drop(index=df[df.Displacement_cc>2480].index, inplace=True)

## 1.2.9 Weight_kg

In [ ]:
# show_distribution(df.Weight_kg)

## 1.2.10 Cons_comb

In [ ]:
# show_distribution(df.cons_comb)

# 1.3 Main statistics

In [ ]:
df.describe().T

In [ ]:
df.corr()

# 1.4 Multicolinearity control

In [ ]:
plt.figure(figsize=(9,9))
sns.heatmap(df.corr(), annot=True)

In [ ]:
df_before_ML = df.copy()

# 2 Data Pre-processing

On of the main component part of the machine learning is to edit the data before proceeding to the implementation of the model. As the last step before model fitting, we need to split the data set as train and test. Then, we should train the model with train data and evaluate the performance of the model on the test data. We can use these train and test data for all algorithms.

We also have to drop our target variable, the column we are trying to predict, price in this case.

We can use many performance metrics for regression to measure the performance of the regression model we train. We can define a function to view different metric results together.

We can also use the cross validation method to measure the estimator performance. Cross validation uses different data samples from our test set and calculates the accuracy score for each data sample. We can calculate the final performance of our estimator by averaging these scores. All these will be performed below for various algorithms.

## 2.1 Split the data

In [ ]:
X= df.drop(columns="price")
y= df.price

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 2.2 Encoding

In [ ]:
df_object.columns

In [ ]:
df.Extras.dtype

In [ ]:
df.Type.value_counts()

In [ ]:
df.Type.value_counts()

In [ ]:
# separate the features as categorical and ordinal features

cat_onehot  = ['make_model', 'body_type', 'vat', 'Fuel', 'Paint_Type', 'Upholstery_type', 'Gearing_Type']
cat_ordinal = ['Type', 'Comfort_Convenience', 'Entertainment_Media', 'Safety_Security']
    
# define the order for each category in ordinal categorical fatures
cat_for_type = ["Used", "Employee's car", "Demonstration", "Pre-registered", "New"]
cat_for_comfort = ['Standard', 'Premium']
cat_for_ent = ['Standard', 'Plus']
cat_for_safety = ['Safety Standard Package', 'Safety Premium Package', 'Safety Premium Plus Package']

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

column_trans = make_column_transformer(
                        (OneHotEncoder(handle_unknown="ignore", sparse=False), cat_onehot), 
                        (OrdinalEncoder(categories= [cat_for_type, cat_for_comfort, cat_for_ent, cat_for_safety]),cat_ordinal),
                         remainder='passthrough', 
                         verbose_feature_names_out=False) 

column_trans=column_trans.set_output(transform="pandas")

In [ ]:
column_trans.fit_transform(X_train).head()

In [ ]:
X_train_trans = column_trans.fit_transform(X_train)
X_test_trans = column_trans.transform(X_test)

In [ ]:
X_train_trans.shape

In [ ]:
X_train_trans.join(y_train).corr()

# check correlations for all features
import plotly.express as px

corr = X_train_trans.join(y_train).corr()
fig = px.imshow(corr,width=1000, height=1000)
fig.show()

In [ ]:
# plt.figure(figsize=(20,15))
# sns.heatmap(X_train_trans.join(y_train).corr(), vmin=-1, vmax=1, cmap="coolwarm");

## 2.3 Scaling

In [ ]:
# our features are mostly between 0 and 1. however engine size, km, price etc have too high values. 
# so we should normalize them by scaling them
scaler = MinMaxScaler().set_output(transform="pandas")
scaler.fit(X_train_trans)

# why min max instead of standardsclaer: bec we have a lot of cat features with values of 0 and 1.
# In order not to change their values, we chose min max scaler

X_train_scaled = scaler.transform(X_train_trans)
X_test_scaled = scaler.transform(X_test_trans)

# 3. Implement Multiple Linear Regression

Steps:
- Import the modul
- Fit the model
- Predict the test set
- Determine feature coefficiant
- Evaluate model performance (use performance metrics for regression and cross_val_score)
- Compare different evaluation metrics

In [ ]:
# user defined function for evaluation metrics

def train_val(model, X_train, y_train, X_test, y_test):
    
    y_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)
    
    scores = {"train": {"R2" : r2_score(y_train, y_train_pred),
                        "mae" : mean_absolute_error(y_train, y_train_pred),
                        "mse" : mean_squared_error(y_train, y_train_pred),
                        "rmse" : mean_squared_error(y_train, y_train_pred, squared=False)},
              "test": {"R2" : r2_score(y_test, y_pred),
                       "mae" : mean_absolute_error(y_test, y_pred),
                       "mse" : mean_squared_error(y_test, y_pred),
                       "rmse" : mean_squared_error(y_test, y_pred, squared=False)}}
    
    return pd.DataFrame(scores)

In [ ]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_scaled, y_train)

In [ ]:
train_val(lm, X_train_scaled, y_train, X_test_scaled, y_test)
# first insight: no overfitting - R2 is high

### 3.1 Adjusted R2 Score

In [ ]:
def adj_r2(y_test, y_pred, X):
    r2 = r2_score(y_test, y_pred)
    n = X.shape[0]   # number of observations
    p = X.shape[1]   # number of independent variables 
    adj_r2 = 1 - (1-r2)*(n-1)/(n-p-1)
    return adj_r2

In [ ]:
# get the predictions to use in the adj_r2 function
y_pred = lm.predict(X_test_scaled)

In [ ]:
adj_r2(y_test, y_pred, X)
# similar to normal R2

### 3.2 Cross validation and overfitting control

In [ ]:
model = LinearRegression()

scores = cross_validate(model, 
                        X_train_scaled, 
                        y_train, 
                        scoring=['r2', 
                                'neg_mean_absolute_error',
                                'neg_mean_squared_error',
                                'neg_root_mean_squared_error'], 
                        cv =5,
                        return_train_score=True)

In [ ]:
pd.DataFrame(scores).iloc[:, 2:].mean()
# no overfitting

In [ ]:
train_val(lm, X_train_scaled, y_train, X_test_scaled, y_test)

### 3.3 Is this data suitable for Linear Regression?

To determine whether a dataset is suitable for linear regression or not, we can perform several checks and analyses on the data. Here are some methods to consider:

Check the correlation between the independent variables and the dependent variable: Linear regression assumes a linear relationship between the independent variables and the dependent variable. We can calculate the correlation coefficients between the independent variables and the dependent variable to check if there is a linear relationship between them. If the correlation is high (close to 1 or -1), it suggests a strong linear relationship, which may be suitable for linear regression. As we have seen above, there are features such as km, hp_kw or age which have higher correlations with the target feature price.

Check for multicollinearity: Multicollinearity occurs when independent variables are highly correlated with each other. In such cases, it becomes difficult to separate the effects of each independent variable on the dependent variable. This can lead to unstable and unreliable regression coefficients. To check for multicollinearity, we can calculate the correlation matrix between the independent variables and check for high correlations between them. As we have seen above, there is no multicolinearity.

Check the residuals: The residuals are the differences between the predicted and actual values of the dependent variable. We can plot the residuals to check for patterns or trends in the data. If the residuals are randomly scattered around zero, it suggests that linear regression may be suitable. However, if there is a pattern in the residuals, such as a U-shape or a curve, it suggests that linear regression may not be appropriate. Below the details.

Check for outliers: Outliers are data points that are significantly different from other data points. They can have a large effect on the regression line and lead to unreliable predictions. We can plot the data and check for any outliers that may need to be removed or dealt with separately. They will be checked below.

Check the distribution of the dependent variable: Linear regression assumes that the dependent variable is normally distributed. We can plot a histogram of the dependent variable to check if it follows a normal distribution. If it is skewed, we may need to transform the data before using linear regression. As if have seen above,the price feature does not have a normal distribution. Therefore we should drop theoutliers below.

### 3.4 Prediction Error with Outliers

In [ ]:
from yellowbrick.regressor import PredictionError
from yellowbrick.features import RadViz

visualizer = RadViz(size=(500, 1000))
model = LinearRegression()
visualizer = PredictionError(model)
visualizer.fit(X_train_scaled, y_train)  # Fit the training data to the visualizer
visualizer.score(X_test_scaled, y_test)  # Evaluate the model on the test data
visualizer.show();

# similar to the insight from boxplot. price over 50k looks like outliers

In [ ]:
y_pred=lm.predict(X_test_scaled)


plt.figure(figsize=(10,7))
plt.subplot(211)

sns.scatterplot(x = y_test, y = y_pred) #-residuals
plt.axhline(y = 0, color ="r", linestyle = "--")
plt.ylabel("y_pred")
plt.xlim([0,30000])
plt.ylim([-1000,30000])

plt.subplot(212)

sns.scatterplot(x = y_test, y = y_pred) #-residuals
plt.axhline(y = 0, color ="r", linestyle = "--")
plt.ylabel("y_pred")
plt.xlim([40000, 80000])
plt.ylim([0,80000])
plt.show();

### Residual plot with poutliers

In [ ]:
from yellowbrick.regressor import ResidualsPlot

visualizer = RadViz(size=(1000, 500))
model = LinearRegression()
visualizer = ResidualsPlot(model)

visualizer.fit(X_train_scaled, y_train)  # Fit the training data to the visualizer
visualizer.score(X_test_scaled, y_test)  # Evaluate the model on the test data
visualizer.show(); 

In [ ]:
y_pred=lm.predict(X_test_scaled)
y_pred_train = lm.predict(X_train_scaled)

residual_test = y_test-y_pred
residual_train = y_train-y_pred_train

plt.figure(figsize=(20,7))
plt.subplot(121)

sns.scatterplot(x = y_pred, y = residual_test, palette="deep", size=2, alpha=0.6)
sns.scatterplot(x = y_pred_train, y = residual_train, size=2, alpha=0.4)

plt.axhline(y = 0, color ="r", linestyle = "--")
plt.ylabel("residuals")
plt.xlabel("y_pred")

plt.subplot(122)

sns.scatterplot(x = y_pred, y = residual_test, palette="deep", size=2, alpha=0.6)
sns.scatterplot(x = y_pred_train, y = residual_train, size=2, alpha=0.4)

plt.axhline(y = 0, color ="r", linestyle = "--")
plt.ylabel("residuals")
plt.xlabel("y_pred")
plt.xlim([1000,10000])
plt.ylim([-10000,20000])
plt.show();

# 4. Filtering the data, cleaning the outliers and rebuilding the model

In [ ]:
df[(df.price > 45000)].shape

In [ ]:
df[(df.price < 7000)].shape

In [ ]:
df.shape

In [ ]:
# since they have a negative effect on the distribution of the data and the performance of Ml algorithms,
# the autos with a price higher than 45000 and lower than 7000 will be dropped and the model will be rebuilt.

df[(df["price"] <7000) | (df["price"] > 45000)].shape

In [ ]:
df.drop(index = df[(df["price"] <7000) | (df["price"] > 45000)].index, inplace=True)

In [ ]:
df.shape

In [ ]:
# user defined function for outliers
def tukey_outliers(data):
    '''
    Identify and optionally drop the outliers from a given dataset using Tukey's method.
    '''

    # Calculate the interquartile range (IQR)
    q1, q3 = np.percentile(data, [25, 75])
    iqr = q3 - q1

    # Calculate the lower and upper bounds for outlier detection
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    # Identify the outliers
    outliers = (data < lower_bound) | (data > upper_bound)
    return lower_bound, upper_bound

In [ ]:
tukey_outliers(df.km)
# we will drop over 100.000 

In [ ]:
# show_distribution(df.km)

In [ ]:
# km feature. drop over 104k
df = df[df.km<100000]
df.shape

In [ ]:
# hp_kW feature
df[df.hp_kW>200].shape

In [ ]:
# drop outliers of hp_kW
df = df[df.hp_kW<200]
df.shape

In [ ]:
# split the new data
X = df.drop(columns = "price")
y = df.price

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# implement encoding via column transformer
X_train= column_trans.fit_transform(X_train)
X_test= column_trans.transform(X_test)

In [ ]:
# scale the data

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# build the model and fit the train data
lm2 = LinearRegression()
lm2.fit(X_train_scaled, y_train)

In [ ]:
# compare the evaluation metrics for train and test data
train_val(lm2, X_train_scaled, y_train, X_test_scaled, y_test)

# we have decreased the errors after cleaning the outliers
# the difference between test and train scores have increased, however we should perform
# cross val to be sure

**Prediction Error without Outliers**


In [ ]:
visualizer = RadViz(size=(500, 1000))
model = LinearRegression()
visualizer = PredictionError(model)
visualizer.fit(X_train_scaled, y_train)  # Fit the training data to the visualizer
visualizer.score(X_test_scaled, y_test)  # Evaluate the model on the test data
visualizer.show();

In [ ]:
model = LinearRegression()
scores = cross_validate(model, 
                        X_train_scaled, 
                        y_train,
                        scoring=['r2', 
                                 'neg_mean_absolute_error',
                                 'neg_mean_squared_error',
                                 'neg_root_mean_squared_error'], 
                        cv=5, 
                        return_train_score=True)

# yeni datamıza göre cross val ile overfiting kontrolü yapıyoruz.

In [ ]:
# check overfitting
scores = pd.DataFrame(scores, index = range(1, 6))
scores.iloc[:,2:].mean()

# train ve validation scores show that there is no overfitting

In [ ]:
y_pred = lm2.predict(X_test_scaled)

lm_R2 = r2_score(y_test, y_pred)
lm_mae = mean_absolute_error(y_test, y_pred)
lm_rmse = mean_squared_error(y_test, y_pred, squared=False)

# scores from the linear model will be used to compare with other algorithms in the end

In [ ]:
pd.DataFrame(lm2.coef_, index = X_train.columns, columns=["Coef"])

# why the coefs of the dummy features are too high?

# Bec of "Dummy variable trap"

# https://geoffruddock.com/one-hot-encoding-plus-linear-regression-equals-multi-collinearity/

# 5. Implement Lasso Regression


- Import the modul
- scale the data or use Normalize parameter as True(If needed)
- Fit the model
- Predict the test set
- Evaluate model performance (use performance metrics for regression)
- Tune alpha hyperparameter by using cross validation and determine the optimal alpha value.
- Fit the model and predict again with the new alpha value.
- Compare different evaluation metrics

In [ ]:
# separate the features as categorical and ordinal features

cat_onehot  = ['make_model', 'body_type', 'vat', 'Fuel', 'Paint_Type', 'Upholstery_type', 'Gearing_Type']
cat_ordinal = ['Type', 'Comfort_Convenience', 'Entertainment_Media', 'Safety_Security']
    
# define the order for each category in ordinal categorical fatures
cat_for_type = ["Used", "Employee's car", "Demonstration", "Pre-registered", "New"]
cat_for_comfort = ['Standard', 'Premium']
cat_for_ent = ['Standard', 'Plus']
cat_for_safety = ['Safety Standard Package', 'Safety Premium Package', 'Safety Premium Plus Package']

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

column_trans = make_column_transformer(
                        (OneHotEncoder(handle_unknown="ignore", sparse=False), cat_onehot), 
                        (OrdinalEncoder(categories= [cat_for_type, cat_for_comfort, cat_for_ent, cat_for_safety]),cat_ordinal),
                         remainder='passthrough', 
                         verbose_feature_names_out=False) 

column_trans=column_trans.set_output(transform="pandas")

In [ ]:
# import Lasso
from sklearn.linear_model import Lasso

In [ ]:
# split the new data
X = df.drop(columns = "price")
y = df.price

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# create and set pipeline
from sklearn.pipeline import Pipeline

operations = [("OneHot_Ordinal_Encoder", column_trans), ("scaler", MinMaxScaler()), ("Lasso", Lasso())]

lasso_model = Pipeline(steps=operations)

lasso_model.fit(X_train, y_train)

In [ ]:
# compare the eval metrics for train and test data
train_val(lasso_model, X_train, y_train, X_test, y_test)

## 5.1. Cross validation - lasso

In [ ]:
operations = [("OneHot_Ordinal_Encoder", column_trans), 
              ("scaler", MinMaxScaler()), 
              ("Lasso", Lasso())]

model = Pipeline(steps=operations)
scores_lasso = cross_validate(model, 
                        X_train, 
                        y_train,
                        scoring=['r2', 
                                 'neg_mean_absolute_error',
                                 'neg_mean_squared_error',
                                 'neg_root_mean_squared_error'],
                        cv=5, 
                        return_train_score=True)

In [ ]:
scores_lasso = pd.DataFrame(scores_lasso, index = range(1, 6))
scores_lasso.iloc[:,2:].mean()
# no overfitting

## 5.2 Finding best alpha for Lasso

In [ ]:
# build the model via pipeline
from sklearn.model_selection import GridSearchCV

operations = [("OneHot_Ordinal_Encoder", column_trans), 
              ("scaler", MinMaxScaler()), 
              ("Lasso", Lasso())]

model = Pipeline(steps=operations)

alpha_space = np.linspace(0.05, 1, 3) # after trying with 0.001,1 and 10 we'll execute with other alpha space

param_grid = {'Lasso__alpha':alpha_space}  # to use in model below.

lasso_grid_model = GridSearchCV(estimator=model,
                          param_grid=param_grid,
                          scoring='neg_root_mean_squared_error',
                          cv=5,
                          n_jobs = -1,
                          return_train_score=True)

In [ ]:
# fit the model
lasso_grid_model.fit(X_train, y_train)

In [ ]:
# what are the best estimators for lasso:
lasso_grid_model.best_estimator_

In [ ]:
lasso_grid_model.best_params_

In [ ]:
# check RMSE for train and test data
pd.DataFrame(lasso_grid_model.cv_results_).loc[lasso_grid_model.best_index_, ["mean_test_score", "mean_train_score"]]

In [ ]:
# compare the eval metrics for train and test data
train_val(lasso_grid_model, X_train, y_train, X_test, y_test)
# similar scores with default lasso

In [ ]:
# assign the metrics to variable to compare with the scores of other algorithms
y_pred = lasso_grid_model.predict(X_test)
lasm_R2 = r2_score(y_test, y_pred)
lasm_mae = mean_absolute_error(y_test, y_pred)
lasm_rmse = mean_squared_error(y_test, y_pred, squared=False)

In [ ]:
# check the coefficients of the features
df_feat_imp = pd.DataFrame(data=lasso_grid_model.best_estimator_["Lasso"].coef_, 
                           index=lasso_grid_model.best_estimator_["OneHot_Ordinal_Encoder"].get_feature_names_out(),
                           columns=["Coef"]).sort_values("Coef")

## 5.3 Modelling with most important features for LASSO

In [ ]:
# visualize feature importance for lasso
plt.figure(figsize=(10,14))
sns.barplot(data= df_feat_imp, 
            x=df_feat_imp.Coef, 
            y=df_feat_imp.index);

In [ ]:
from yellowbrick.model_selection import FeatureImportances
from yellowbrick.features import RadViz

model = lasso_grid_model.best_estimator_["Lasso"]

viz = FeatureImportances(model, 
                         labels=lasso_grid_model.best_estimator_["OneHot_Ordinal_Encoder"].get_feature_names_out())

visualizer = RadViz(size=(720, 3000))
viz.fit(X_train, y_train)
viz.show();


**create a new dataset with the most important features for lasso**

In [ ]:
# create new df according to the most important features
df_new = df[["make_model", "age","hp_kW", "km", "Type", "Gearing_Type", "price"]]


In [ ]:
# preprocessing
X = df_new.drop(columns = ["price"])
y = df_new.price

In [ ]:
# split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# separate the features as categorical and ordinal features

cat_onehot  = ["make_model", "Gearing_Type"]
cat_ordinal = ['Type']
    
# define the order for each category in ordinal categorical fatures
cat_for_type = ["Used", "Employee's car", "Demonstration", "Pre-registered", "New"]

column_trans = make_column_transformer((OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_onehot),
                                       (OrdinalEncoder(categories= [cat_for_type]), cat_ordinal),
                                        remainder='passthrough',
                                        verbose_feature_names_out=False)

In [ ]:
operations = [("OneHot_Ordinal_Encoder", column_trans), 
              ("scaler", MinMaxScaler()), 
              ("Lasso", Lasso(alpha=1))] # with best alpha

lasso_final_model = Pipeline(steps=operations)

lasso_final_model.fit(X_train, y_train)
train_val(lasso_final_model, X_train, y_train, X_test, y_test)


In [ ]:
# cross validation
operations = [("OneHot_Ordinal_Encoder", column_trans), 
              ("scaler", MinMaxScaler()), 
              ("Lasso", Lasso(alpha=1))]

model = Pipeline(steps=operations)

scores_lasso_final_model = cross_validate(model, X_train, y_train,
                        scoring=['r2', 
                                 'neg_mean_absolute_error',
                                 'neg_mean_squared_error',
                                 'neg_root_mean_squared_error'],
                        cv=5, 
                        return_train_score=True)

In [ ]:
scores_lasso_final_model = pd.DataFrame(scores_lasso_final_model, index = range(1, 6))
scores_lasso_final_model.iloc[:,2:].mean()
# no overfitting

In [ ]:
# assign metrics to variables
y_pred = lasso_final_model.predict(X_test)
fm_R2 = r2_score(y_test, y_pred)
fm_mae = mean_absolute_error(y_test, y_pred)
fm_rmse = mean_squared_error(y_test, y_pred,squared=False)

# 6. RANDOM FOREST

In [ ]:
X=df.drop("price", axis=1)
y=df.price

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=5)

In [ ]:
# modelling with pipeline
cat = X.select_dtypes("object").columns
cat

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

ord_enc = OrdinalEncoder(handle_unknown='use_encoded_value', 
                         unknown_value=-1)

column_trans = make_column_transformer((ord_enc, cat), 
                                        remainder='passthrough',
                                        verbose_feature_names_out=False).set_output(transform="pandas")

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline


operations = [("OrdinalEncoder", column_trans), 
              ("RF_model", RandomForestRegressor(random_state=101))]

pipe_model = Pipeline(steps=operations)

pipe_model.fit(X_train, y_train)

In [ ]:
train_val(pipe_model, X_train, y_train, X_test, y_test)
# default RF performs very better than lin reg or lasso

In [ ]:
# check overfitting via cv 
operations = [("OrdinalEncoder", column_trans), 
              ("RF_model", RandomForestRegressor(random_state=101))]

model = Pipeline(steps=operations)

scores = cross_validate(model, 
                        X_train, 
                        y_train, 
                        scoring=['r2',
                                 'neg_mean_absolute_error',
                                 'neg_mean_squared_error',
                                 'neg_root_mean_squared_error'],
                        cv =5,
                        return_train_score=True)
df_scores = pd.DataFrame(scores)
df_scores.mean()[2:]

# error in test data is higer; R2 is lower for test data. overfit. 

In [ ]:
# gridsearch to avoid overfitting
operations = [("OrdinalEncoder", column_trans),
              ("RF_model", RandomForestRegressor(random_state=101))]

model = Pipeline(steps=operations)

In [ ]:
param_grid = {"RF_model__criterion":["squared_error", "absolute_error"],  # "poisson"
              "RF_model__n_estimators":[250, 500],
              "RF_model__max_depth": [1, 2],
              "RF_model__min_samples_leaf": [1, 2, 3],  # values lower than 3 to avoid overfitting
              "RF_model__min_samples_split": [2, 3, 5],
              "RF_model__max_features":[1.0, X.shape[1]/3, 6]}  # None, auto, 1.0 all same

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_model = GridSearchCV(estimator=model,
                          param_grid=param_grid,
                          scoring='neg_root_mean_squared_error',
                          cv=5,
                          n_jobs=-1,
                          return_train_score=True)
grid_model.fit(X_train,y_train)

In [ ]:
grid_model.best_estimator_

In [ ]:
pd.DataFrame(grid_model.cv_results_).loc[grid_model.best_index_, ["mean_test_score", "mean_train_score"]]


In [ ]:
train_val(grid_model, X_train, y_train, X_test, y_test)

In [ ]:
# cross val
operations = [("OrdinalEncoder", column_trans), 
              ("RF_model", RandomForestRegressor(max_depth=8, 
                                                 max_features=6, 
                                                 min_samples_leaf=3,
                                                 n_estimators=350,
                                                 random_state=101))]

model = Pipeline(steps=operations)

scores = cross_validate(model,
                        X_train, 
                        y_train, 
                        scoring=['r2',
                                 'neg_mean_absolute_error',
                                 'neg_mean_squared_error',
                                 'neg_root_mean_squared_error'], 
                        cv = 5,
                        return_train_score=True)

df_scores = pd.DataFrame(scores, index=range(1,6))
df_scores.mean()[2:]

**Feature Importance**

In [ ]:
# IMPORTANT: with the best hyperparameters from gridsearchcv
operations = [("OrdinalEncoder", column_trans), 
              ("RF_model", RandomForestRegressor(max_depth=8,
                                                 max_features=6,
                                                 min_samples_leaf=3, 
                                                 n_estimators=350,
                                                 random_state=101))]

pipe_model = Pipeline(steps=operations)

pipe_model.fit(X_train, y_train)

In [ ]:
features = pipe_model["OrdinalEncoder"].get_feature_names_out()
features

In [ ]:
df_f_i = pd.DataFrame(data=pipe_model["RF_model"].feature_importances_, 
                      index=features,
                      columns=["Feature Importance"])
df_f_i = df_f_i.sort_values("Feature Importance", ascending=False)
df_f_i

**Feature selection**

In [ ]:
X2 = X[["hp_kW", "age", "Gears", "make_model", "km"]]
X2.head()

# en kuvvetli ilk 5 feature seçiyoruz.

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X2, y, test_size=0.2, random_state=5)

# Yeni datamıza göre train ve test datamızı tekrar belirliyoruz.

In [ ]:
cat2 = ["make_model"]

ord_enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

column_trans = make_column_transformer((ord_enc, cat2), 
                                        remainder='passthrough',
                                        verbose_feature_names_out=False).set_output(transform="pandas")


operations = [("OrdinalEncoder", column_trans),
              ("RF_model", RandomForestRegressor(max_depth=8, # best hyperparameters from gridsearch
                                                 max_features=5, 
                                                 min_samples_leaf=3, 
                                                 n_estimators=350,
                                                 random_state=101))]

pipe_model = Pipeline(steps=operations)
pipe_model.fit(X_train,y_train)
train_val(pipe_model, X_train, y_train, X_test, y_test)

# 5 feature ile benzer skorlar aldık.

In [ ]:
operations = [("OrdinalEncoder", column_trans),
              ("RF_model", RandomForestRegressor(max_depth=8,
                                                 max_features=5, 
                                                 min_samples_leaf=3, 
                                                 n_estimators=350,
                                                 random_state=101))]

model = Pipeline(steps=operations)
scores = cross_validate(model,
                        X_train, 
                        y_train, 
                        scoring=['r2',
                                 'neg_mean_absolute_error',
                                 'neg_mean_squared_error',
                                 'neg_root_mean_squared_error'], 
                        cv = 5,
                        return_train_score=True)

df_scores = pd.DataFrame(scores, index=range(1,6))
df_scores.mean()[2:]

# no overfitting

In [ ]:
# avg error for test data
2088 / np.mean(y)

In [ ]:
# avg error for train data
1921/np.mean(y)

# 6. XGBOOST

In [ ]:
df.to_csv('for_xgboost.csv', index=False)

In [ ]:
X=df.drop("price", axis=1)
y=df.price

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=5)

In [ ]:
# modelling with pipeline
cat = X.select_dtypes("object").columns
cat

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

ord_enc = OrdinalEncoder(handle_unknown='use_encoded_value', 
                         unknown_value=-1)

column_trans = make_column_transformer((ord_enc, cat), 
                                        remainder='passthrough',
                                        verbose_feature_names_out=False).set_output(transform="pandas")


In [ ]:
from xgboost import XGBRegressor

operations = [("OrdinalEncoder", column_trans), 
              ("XGB_model", XGBRegressor(random_state=101))]

pipe_model = Pipeline(steps=operations)

pipe_model.fit(X_train, y_train)

In [ ]:
train_val(pipe_model, X_train, y_train, X_test, y_test)
# trains and test rmse : overfitting

In [ ]:

# cross val
operations = [("OrdinalEncoder", column_trans), 
              ("XGB_model", XGBRegressor(random_state=101))]

model = Pipeline(steps=operations)

scores = cross_validate(model,
                        X_train,
                        y_train, 
                        scoring=['r2', 
                                 'neg_mean_absolute_error',
                                 'neg_mean_squared_error',
                                 'neg_root_mean_squared_error'],
                        cv =5,
                        return_train_score=True)
pd.DataFrame(scores).iloc[:, 2:].mean()

# overfiting

### Gridsearch for XGBoost

In [ ]:
param_grid = {"XGB_model__n_estimators":[75,100,150],
              "XGB_model__max_depth":[6,7], 
              "XGB_model__learning_rate": [0.09, 0.1],
              "XGB_model__subsample": [0.8, 0.9], 
              "XGB_model__colsample_bytree":[0.8],
              "XGB_model__colsample_bylevel":[1]
              }

In [ ]:
operations = [("OrdinalEncoder", column_trans),
              ("XGB_model", XGBRegressor(random_state=101))]

model = Pipeline(steps=operations)

grid_model = GridSearchCV(estimator=model,
                          param_grid=param_grid,
                          scoring='neg_root_mean_squared_error',
                          cv=5,
                          n_jobs=-1,
                          return_train_score=True)

In [ ]:
grid_model.fit(X_train, y_train)

In [ ]:
grid_model.best_params_

In [ ]:
grid_model.best_estimator_

In [ ]:
pd.DataFrame(grid_model.cv_results_).loc[grid_model.best_index_, ["mean_test_score", "mean_train_score"]]

# overfiting

In [ ]:
prediction = grid_model.predict(X_test)

xgb_rmse = mean_squared_error(y_test, prediction, squared=False)

train_val(grid_model, X_train, y_train, X_test, y_test)

# rmse difference is too high for test and train data

**Feature importance**

In [ ]:
operations = [("OrdinalEncoder", column_trans), 
              ("XGB_model", XGBRegressor(n_estimators=150,
                                         learning_rate=0.09, 
                                         max_depth=7,
                                         subsample=0.8, 
                                         random_state=101))]

pipe_model = Pipeline(steps=operations)

pipe_model.fit(X_train, y_train)

In [ ]:
features

In [ ]:
imp_feats = pd.DataFrame(data=pipe_model["XGB_model"].feature_importances_,
                         columns=['xgb_Importance'],
                         index=features)

xgb_imp_feats = imp_feats.sort_values('xgb_Importance', ascending=False)
xgb_imp_feats

In [ ]:
ax = sns.barplot(data=xgb_imp_feats, 
                 x=xgb_imp_feats.index, 
                 y='xgb_Importance')

ax.bar_label(ax.containers[0],fmt="%.3f")
plt.xticks(rotation=90);

In [ ]:
df_xg = df[["hp_kW", "age","make_model", "Gears", "Gearing_Type", "price"]]

In [ ]:
X=df_xg.drop("price", axis=1)
y=df_xg.price

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=5)

In [ ]:
# modelling with pipeline
cat = X.select_dtypes("object").columns
cat

In [ ]:
ord_enc = OrdinalEncoder(handle_unknown='use_encoded_value', 
                         unknown_value=-1)

column_trans = make_column_transformer((ord_enc, cat), 
                                        remainder='passthrough',
                                        verbose_feature_names_out=False).set_output(transform="pandas")

In [ ]:
operations = [("OrdinalEncoder", column_trans), 
              ("XGB_model", XGBRegressor(n_estimators=150,
                                         learning_rate=0.09, 
                                         max_depth=6,
                                         subsample=0.8, 
                                         random_state=101))]

pipe_model = Pipeline(steps=operations)

pipe_model.fit(X_train, y_train)

In [ ]:
train_val(pipe_model, X_train, y_train, X_test, y_test)
# no overfitting

In [ ]:
# cross val
operations = [("OrdinalEncoder", column_trans), 
              ("XGB_model", XGBRegressor(n_estimators=150,
                                         learning_rate=0.09, 
                                         max_depth=6,
                                         subsample=0.8, 
                                         random_state=101))]

model = Pipeline(steps=operations)

scores = cross_validate(model,
                        X_train,
                        y_train, 
                        scoring=['r2', 
                                 'neg_mean_absolute_error',
                                 'neg_mean_squared_error',
                                 'neg_root_mean_squared_error'],
                        cv =5,
                        return_train_score=True)
pd.DataFrame(scores).iloc[:, 2:].mean()


In [ ]:
1900/np.mean(y)
# test data, avg error

In [ ]:
1794 / np.mean(y)
# avg error for train data

# 7. Final MOdel

Random Forest with 5 features

In [ ]:
X=df_xg.drop("price", axis=1)
y=df_xg.price

In [ ]:
# modelling with pipeline
cat = X.select_dtypes("object").columns

ord_enc = OrdinalEncoder(handle_unknown='use_encoded_value', 
                         unknown_value=-1)

column_trans = make_column_transformer((ord_enc, cat), 
                                        remainder='passthrough',
                                        verbose_feature_names_out=False).set_output(transform="pandas")

operations = [("OrdinalEncoder", column_trans), 
              ("XGB_model", XGBRegressor(n_estimators=150,
                                         learning_rate=0.09, 
                                         max_depth=6,
                                         subsample=0.8, 
                                         random_state=101))]

final_model = Pipeline(steps=operations)

final_model.fit(X, y)

In [ ]:
final_model["OrdinalEncoder"].fit_transform(X).head()

In [ ]:
X

## 7.1 Prediction

In [ ]:
df_xg.groupby(["make_model", "Gearing_Type"]).mean()

In [ ]:
samples = {"make_model": ["Opel Astra", "Renault Clio"],
           "Gearing_Type": ["Automatic", "Manual"],
           "Gears":[6, 5],
           'age':[1,1],
           'hp_kW': [100, 50]}

In [ ]:
df_samples = pd.DataFrame(samples)
df_samples

In [ ]:
final_model.predict(df_samples)

## Pickle the final model

In [ ]:
import pickle

In [ ]:
filename = "final_model"
pickle.dump(final_model, open(filename, "wb"))

In [ ]:
# read the pickle model
final_model_p = pickle.load(open("final_model", "rb"))

In [ ]:
prediction = final_model_p.predict(samples)
prediction

In [ ]:
df_xg.reset_index(drop=True, inplace=True)

In [ ]:
df_xg.to_csv('last_data.csv', index=False)